## Welcome to the Unified Powered Guidance for landing!
This notebook will guide you to perform a bolza landing, that is, just land near the target location rather than precisely on the target.  
If you just want to land without getting deeper in the algorithm, just fill the parameters in cell 3 and run all cells.  
First, let's import the packages.

In [1]:
import time
import logging

import krpc
import numpy as np
from IPython.display import clear_output

from UPG import UPG, Status
import utils
import Apollo


Next, we will need to build up telemetry for the landing craft.

In [2]:
conn = krpc.connect()
space_center = conn.space_center
vessel = space_center.active_vessel
ap = vessel.auto_pilot
body = vessel.orbit.body
body_frame = body.reference_frame
flight = vessel.flight(body_frame)

# body constants
r_e = body.equatorial_radius
g_0 = body.surface_gravity

Then, specify the landing parameters. The coefficient `k`  decides the precision of the landing. Set it to 0 means you don't care about the precision, and the craft will just land at some place around the target. Basically, the bigger `k` means more fuel and the better landing precision.

In [3]:
lon = -60.0
lat = 0.0
final_velocity = 5.    # The target velocity for UPG. Note that is this not the touch down velocity.
final_height = 10. # The target height above target when upg ends.
min_throttle = 0.1  # Minimum throttle. This must be exactly equal to your engine's min throttle.
# min_thrust = 0.
k = 500.   # Coefficient for bolza landing.
t_1 = 10.    # First burn time, in secondes.
rated_burn_time = 800   # Engine's rated burn time, or the time you want to burn at most.

mode = ['bolza', 'pinpoint'][1]

# if you find your craft turning too slow, alter the following values.
ap.stopping_time = (.9, .9, .9)
# ap.deceleration_time = (3., 3., 3.)
ap.attenuation_angle = (0.8, 0.8, 0.8)

Next we will build a target frame whose origin is the target location, and acquire some useful vectors.

In [4]:
# target vector, with final height added.
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height + final_height, body_frame))

# target reference frame.
target_frame = utils.target_reference_frame(space_center, body, lat, lon)
# get final velocity and up vector in body frame
up = r_target / np.linalg.norm(r_target)
v_target = -final_velocity * up

# draw the target
line = conn.drawing.add_line((0., 0., 0.), (20., 0., 0.), target_frame)
line.thickness = 2

...And the data streams to continuously send data to the program.

In [5]:
vacuum_specific_impulse = conn.add_stream(getattr, vessel,
                                          'vacuum_specific_impulse')
max_thrust = conn.add_stream(getattr, vessel, 'available_thrust')
current_thrust = conn.add_stream(getattr, vessel, 'thrust')
mass = conn.add_stream(getattr, vessel, 'mass')
situation = conn.add_stream(getattr, vessel, 'situation')
ut = conn.add_stream(getattr, space_center, 'ut')
height = conn.add_stream(getattr, flight, 'surface_altitude')
mean_altitude = conn.add_stream(getattr, flight, 'mean_altitude')
speed = conn.add_stream(getattr, flight, 'speed')
velocity = conn.add_stream(vessel.velocity, body_frame)
position = conn.add_stream(vessel.position, body_frame)

Now it's time to set up the guidance program.

In [6]:
# get initial values
v_0 = utils.vector(velocity())
r_0 = utils.vector(position())
# initial guess
pv = -v_0 / np.linalg.norm(v_0)
pr = np.zeros((3, 1))
t_f = np.array([[rated_burn_time]]) / 2

print("Initiating UPG")
upg = UPG(r_0=r_e, g_0=g_0, r_target=r_target, v_target=v_target, r_t=r_0, v_t=v_0,
    mass=mass(), max_thrust=max_thrust(), min_throttle=min_throttle, 
    specific_impulse=vacuum_specific_impulse(), k=k, t_0=ut(), t_1=t_1, p_r_0=pr, p_v_0=pv,
    t_f=t_f, t_2_bound=[5, rated_burn_time / 2])

print("Evaluating t2...")
upg.solve_t_2()
print("Optimal t2: {:.2f} s".format(upg.get_t_2))

logging.basicConfig(filename='upg.log',
                    filemode='w',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

Initiating UPG
Evaluating t2...
Terminate program if it stucks at solving t_2.
Optimal t2: 379.73 s


We have initialize the UPG, now just wait until it's time to start descent. This is done by continuously solving soft landing and check if the final location is at the target.

To ensure guidance precision, the next phase is also included in this cell. It will

In [7]:
# PDI determination
print("Waiting for powered descent initial")
ap.reference_frame = body_frame
ap.engage()
guidance_interval = 0.25


while True:
    v_0 = utils.vector(velocity())
    r_0 = utils.vector(position())
    upg.update_state(r_t=r_0, v_t=v_0, mass=mass(),
               max_thrust=max_thrust(), t_0=ut())
    if upg.status == Status.PDI:
        upg.update_start_time(ut())
        upg.solve(mode='soft')
        r_guidance = utils.downrange(r_0, upg.r_final, body, body_frame)
        r_togo = utils.downrange(r_0, r_target, body, body_frame)
        print("guidance downrange: {0:,.2f}m, target downrange: {1:,.2f}m"
            .format(r_guidance, r_togo))
        # start descent when overshoot the target with margin of 1km.
        if r_guidance > r_togo + 1000.:
            print("UPG engaged")
            upg.status = Status.Powered_descent
            continue
        # the direction of soft landing may be different from the bolza one. slove the bolza problem again to give the correct direction.
        upg.solve(mode=mode)

    elif upg.status == Status.Powered_descent:
        upg.solve(mode=mode)
        t_1_go = upg.t_1_go
        t_2_go = upg.t_2_go
        distance = np.linalg.norm(r_0 - upg.r_final)
        v_go = upg.v_go
        print("t_togo: {:.2f} s".format(upg.t_f), end=", ")
        if t_1_go > 0:
            print("{:.2f} s to throttle back".format(t_1_go))
        elif t_2_go > 0:
            print("{:.2f} s to full throttle".format(t_2_go))

        print("distance: {:,.2f} m".format(distance), end=", ")
        print("v_togo: {:,.2f} m/s".format(v_go), end=", ")
        print("Landing error: {:.2f} m".format(np.linalg.norm(upg.r_final - r_target)))
        print("Convergence: " + str(upg.convergence) + " norm: {}".format(upg.norm))
        if not upg.convergence:
            print("Last convergence: {:.2f} s ago".format(upg.last_convergence))
            print("Last error: " + upg.last_err_msg)
            logging.info("Failed to converge, the results are:\n" + str(upg.fun))
        # correct target to the correct height as the distance to target could be large.
        if k == 0.:
            r_target = utils.move_position2height(
                final_height, upg.r_final, body, body_frame)
            up = r_target / np.linalg.norm(r_target)
            v_target = -final_velocity * up
            upg.set_target(r_target, v_target)

        vessel.control.throttle = upg.throttle
        downrange = utils.downrange(r_0, r_target, body, body_frame)
        print("{:,.2f} m to terminal guidance".format(downrange - 3000))
        if downrange < 3000. or upg.t_f < 60.:
            print("UPG disengaged")
            break
        if upg.last_convergence > 30.:
            if mode == 'pinpoint':
                mode = 'bolza'
            else:
                break
    ap.target_direction = upg.thrust_direction
    clear_output(wait=True)
    time.sleep(guidance_interval)

# terminal guidance: use apollo guidance algorithm.
# change target to ground
if k != 0. or mode == 'pinpoint':
    r_target = utils.vector(body.position_at_altitude(
        lat, lon, body.surface_height(lat, lon) + final_height, body_frame))
if k == 0. or np.arctan2((mean_altitude() - target_height), utils.downrange(r_target, upg.r_final, body, body_frame)) <\
    np.arcsin(-flight.vertical_speed / speed()):
# make another landing frame with the predicted final location
    r_target = utils.move_position2height(
        final_height , upg.r_final, body, body_frame)
    up = r_target / np.linalg.norm(r_target)
    lat = body.latitude_at_position(tuple(r_target.flatten()), body_frame)
    lon = body.longitude_at_position(tuple(r_target.flatten()), body_frame)
    target_height = body.surface_height(lat, lon)
    target_frame = utils.target_reference_frame(space_center, body, lat, lon)
    line = conn.drawing.add_line((0., 0., 0.), (20., 0., 0.), target_frame)
    line.thickness = 3

a_f = 1.5# final acceleration 2g(g with respect to body)
v_f = 3.   # touch down velocity.

# calculate t_go
v_0 = speed()
sin_gamma = flight.vertical_speed / v_0
h_0 = mean_altitude() - target_height
a_t = Apollo.a_t(v_0, sin_gamma, h_0, g_0)
t_f = Apollo.t_togo(v_0, sin_gamma, a_t, g_0)
t_f = 1.1 * upg.t_f
print('tgo: {:.2f}'.format(t_f))
t_f = t_f + ut()    # convert to ut
apdg = Apollo.APDG(r_target, v_f, a_f, t_f, ut(), up, mass(), r_0, v_0, g_0, 8.)
print("Terminal guidance engaged")
while True:
    v_0 = utils.vector(velocity())
    r_0 = utils.vector(position())
    apdg.update(r_0, v_0, mass(), ut())
    apdg.compute()

    vessel.control.throttle = utils.thrust2throttle(
    apdg.thrust, max_thrust(), min_throttle)
    ap.target_direction = apdg.thrust_direction

    print("t_go: {:.2f}".format(apdg.t_go), end=' ')
    r_go = np.linalg.norm(r_target - r_0)
    print("distance to land: {:.2f}".format(r_go))
    print("thrust error: {:.2f} kN".format((vessel.thrust - apdg.thrust) / 1e3))

    print('AP pitch:{0:2f} AP heading:{1:.2f}'
          .format(ap.target_pitch, ap.target_heading))
    print('pitch:{0:2f} heading:{1:.2f}'
          .format(flight.pitch, flight.heading))
    print("AP direction error {:.2f}".format(ap.error))
    
    clear_output(wait=True)
    if apdg.t_go < 0.5 or r_go < 10 or height() < 10:
        print("APDG disengaged")
        break
    
    time.sleep(guidance_interval)

# constant descent
ap.target_direction = tuple(up.flatten())
# vessel.control.throttle = utils.thrust2throttle(
#     1.1 * mass() * g_0, max_thrust(), min_throttle)


while True:
    # vessel.control.throttle += 0.005 * (speed() - v_f)
    if situation() == space_center.VesselSituation.landed \
            or situation() == space_center.VesselSituation.splashed\
            or flight.vertical_speed > -0.5:
        print("landed!")
        break
ap.disengage()
vessel.control.throttle = 0.
ap.sas = True

r_0 = utils.vector(position())
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height, body_frame))
print("Landing precision: {:.2f}m".format(np.linalg.norm(r_0 - r_target)))


APDG disengaged
landed!
Landing precision: 5.57m
